In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import *
from holoviews import streams

import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils
from config import Config

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

In [ ]:
class Curve():
    """
    Explicit Curve representation using parametrization 
    
    #- xfunc
    #- yfunc
    - data: ndarray of shape n_points x 2 
    - n_points: number of sampled points on the curve)
    - ps: curve parameter ps, 1-dimensional 

    ---
    - set_data(self, new_data):
    - reset(self):
        self.set_data(np.zeros_like(curve's npoints)
    
    - compute_tangents(self):
        
    - compute_normals(self):
    
    """
    
    ################################################################################
    # Initialization
    ###############################################################################
    def __init__(self, ps, data):
        self.n_points = len(ps)
        self.data = self.to_df(ps,data)
        
    @classmethod
    def fromPoints(cls, ps, points):
        """
        Args:
        - points (2dim ndarray): two dimensional ndarray of points. Each column corresponds
        to x and y coordinates
        """
        return cls(ps, points)
    
    @classmethod
    def fromFunctions(cls, ps, xfunc, yfunc):
        """
        Args:
        - xfunc (callable): given input of a float or a list of floats,
            returns a list of float(s) representing the xcoordinate(s)
        - yfunc (callable): similar to xfunc, but returns ycoordinate(s)
        
        """
        data = np.c_[xfunc(ps), yfunc(ps)]
        return cls(ps, data)
    
    @classmethod
    def fromDF(cls, df):
        """
        Args:
        - df (pd.DataFrame): has information on the parameter space and xy coordinates 
            - must have three columns representing 'p','x','y'
        """
        if df.shape[1] != 3:
            raise ValueError (f"""
            Input df must have three columns representing the parameter p, xcoord, and ycoord:
            {df.shape[1]}
            """)
        ps, data = df.iloc[:,0], df.iloc[:,[1,2]]
        return cls(ps, data)
    
    @classmethod
    def fromConfig(cls, config, data):
        """
        Args:
        - config (dict or Config)
            - keys: n_points, prange
        """
        ps = np.linspace(*config.p_range, num=config.n_points)
        return cls(ps, data)
    
    @staticmethod
    def to_df(ps, data):
        if isinstance(ps, pd.DataFrame):
            ps = ps.to_numpy()
        if isinstance(data, pd.DataFrame):
            data = data.to_numpy()
        return pd.DataFrame({'p': ps, 'x': data[:,0], 'y':data[:,1]})
        
    ################################################################################
    # Tangents and Normals
    ###############################################################################
    def compute_tangents(self):
        """
        Estimate tangent vectors \vec{t}(p) at each parameter p's values
        using central finite difference 
            - assumes neighboring p values are mapped to neighboring points
            on the curve
        """
        pass
    
    def compute_normals(self):
        """
        Estimate normal vectors based on the computed tangent vectors 
        by rotating tangents 90 degrees counterclockwise
        """
        pass

    ###############################################################################
    # Visualization
    ###############################################################################
    def hvplot(self):
        #hv.Points
        return (
            hv.Points(self.data).opts(padding=0.1, width=300, height=300, size=3, color='black')
        )
    def reset(self):
        pass
    

In [ ]:
%opts Points [tools=['hover'], active_tools=['wheel_zoom']]

In [ ]:
# Test1: Curve(ps, points) or Curve.fromPoints(ps, points)
def test_curve_fromPoints():
    ps = np.linspace(0,1,5)
    points = np.array([(0,0),(0,1),(1,0),(1,-1),(-1,1)])
    c1 = Curve(ps,points)
    c2 = Curve.fromPoints(ps, points)
    display(c1.hvplot())
    display(c2.hvplot())
# test_curve_fromPoints()

In [ ]:
# Test2: Curve.fromeFunctions(ps, xfunc, yfunc)
def test_curve_fromFunctions():
    ps = np.linspace(0,1,5)
    xfunc = lambda p: p
    yfunc = lambda p: [0]*len(p)
    
    c = Curve.fromFunctions(ps,xfunc, yfunc)
    display(c.hvplot())
# test_curve_fromFunctions()

In [ ]:
# Test3: Curve.fromDF(df)
def test_curve_fromDF():
    ps = np.linspace(0,1,5)
    points = np.array([(0,0),(0,1),(1,0),(1,-1),(-1,1)])
    df = pd.DataFrame({'p': ps, 'x': points[:,0], 'y':points[:,1]})
    c = Curve.fromDF(df)
    display(c.hvplot())
# test_curve_fromDF()

In [ ]:
# Test4: Curve.fromConfig
## c = Curve(config, data)
def test_curve_fromConfig():
    from config import Config

    ps = np.linspace(0,1,5)
    ps_config = Config(n_points=5,p_range=(0,2*np.pi))
    points = np.array([(0,0),(0,1),(1,0),(1,-1),(-1,1)])
    df = pd.DataFrame({'p': ps, 'x': points[:,0], 'y':points[:,1]})
    c = Curve.fromConfig(ps_config, points)
    c.show()
# test_curve_fromConfig()

In [ ]:
class CurveGen(param.Parametrized):
    """
    Explicit Curve representation using parametrization 
    
    #- xfunc
    #- yfunc
    - data (n_points by 2 ndarray)
    - n_points (number of sampled points on the curve)
    - p (curve parameter)'s range
    
    ---
    - set_data(self, new_data):
    - reset(self):
        self.set_data(np.zeros_like(curve's npoints)
    
    - compute_tangents(self):
        
    - compute_normals(self):
    
    """
    p_range = param.Range(default=(0,1), label="parameter p's range")
    n_points = param.Integer(default=100, label='Number of sampled points on the curve')
    p = param.Selector(objects=np.linspace(*p_range, num=n_points.default))
    
    @param.depends('n_points', 'prange')
    def _reset_param_p(self):
        p.objects = np.linspace(*self.p_range, num=self.n_points)
                       
                       
    ################################################################################
    # Initialization
    ###############################################################################
    def __init__(self):
        super().__init__() #(*args, **kwargs)?
        ps = p.objects
        curve = Curve(ps,[])
        
        pass
    
    def compute_tangents(self):
        """
        Estimate tangent vectors \vec{t}(p) at each parameter p's values
        using central finite difference 
            - assumes neighboring p values are mapped to neighboring points
            on the curve
        
        """
        pass
    
    def compute_normals(self):
        """
        Estimate normal vectors based on the computed tangent vectors 
        by rotating tangents 90 degrees counterclockwise
        """
        pass
    
    def show(self):
        #hv.Points
        pass
    def reset(self):
            

In [ ]:
class CurveViewer(param.Parametrized):
    

In [ ]:
ps = np.linspace(0,1,5)
points = np.array([(0,0),(0,1),(1,0),(1,-1),(-1,1)])
df = pd.DataFrame({'p': ps, 'x': points[:,0], 'y':points[:,1]})
c = Curve.fromDF(df)

hv.DynamicMap(lambda p: c.hvplot() * hv.Points( [

In [ ]:
class CurveMover(param.Parametrized):
    """
    
    """
    def __init__(self):
        """
        curves (list): n_curves by n_points (ie. parameter p's resolution)
        time (float):
        
        """
        super().__init__()
        pass
    
    def move(self, curve, V):
        """
        V: external velocity field at each point on the curve
         - V is a function of p, the curve parameter
         - or a ndarray of the same shape as any curve c's c.data,
             ie. self.curves.shape[1]
        """
        if V is callable: 
            evaluate V at each point p
        return curve.clone().set_data(curve.data + V) #check if .clone() is the right function in param.Parametrized 
    
            
        


In [ ]:
# Tensorboard
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ../logs/
